In [42]:

import numpy as np
import matplotlib.pyplot as plt
import os
import platform
import sys
import os 

sys.path.append("C:\\Program Files\\Lumerical\\v242\\api\\python") # this line is to direct the python script ot the lumerical python api 
import lumapi

current_directory= os.getcwd()
current_directory= r"{}".format(current_directory)
sys.path.append(current_directory)

focal_length = 100.0e-6
LensDiameter= 20.0e-6
SubstrateIndex= 1.46 
PillarIndex= 3.4757
period = 0.4e-6 
wavelength = 1.55e-6

unit_cell_library_file_name= 'EH_and_phase_vs_radius_interp_fdtd_si_sio2_sept13.mat'
zemax_phase_file_name=  'output_x_y_rad_apexPHAZE_optimized.txt'

In [90]:
fdtd= lumapi.FDTD()

setup_script= """
# create an aperture matching the diameter of the metalens 
R = getnamed("metalens","lens_radius");
W = getnamed("metalens::substrate","x span");
V = R*exp(1i*linspace(0,2*pi,200));
V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
setnamed("aperture","vertices",V);
"""
fdtd.set("setup script",  setup_script)

radius_phase_file = current_directory+ '\\2_meta_library\\unit cell'+'\\' + unit_cell_library_file_name
zfile= current_directory+ '\\'+ zemax_phase_file_name

# #==================================================
# Add FDTD
# #==================================================
fdtd.addfdtd()
fdtd.set("dimension",2);  #  1 = 2D, 2 = 3D
fdtd.set("x",0)
fdtd.set("x span",25.0e-6)
fdtd.set("y",0)
fdtd.set("y span",25.0e-6)
fdtd.set("z",0.6e-6)
fdtd.set("z span",1.8e-6)
fdtd.set("mesh type", "auto non-uniform")
fdtd.set("mesh accuracy", 3)



# create an aperture matching the diameter of the metalens 
# myscript= """
# R = getnamed("metalens","lens radius");
# W = getnamed("metalens::substrate","x span");
# V = R*exp(1i*linspace(0,2*pi,200));
# V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
# setnamed("aperture","vertices",V);
# """
# fdtd.set("script",myscript)

# #==================================================
# Add the metalens geometry 
# #==================================================
fdtd.addstructuregroup()
fdtd.set("name", "metalens")
fdtd.adduserprop("radius_vs_phase_data_file", 1, radius_phase_file)
fdtd.adduserprop("zemax_optimized_phase_file", 1,zfile)
fdtd.adduserprop("focal_length", 2, focal_length)
fdtd.adduserprop("lens_radius", 2, LensDiameter/2)
fdtd.adduserprop("substrate_index", 0, SubstrateIndex)
fdtd.adduserprop("pillar_index", 0, PillarIndex)



myscript =f"""deleteall;
matlabload(radius_vs_phase_data_file);
?mat_sub;
np = round({LensDiameter/2}/ {period});
x_mask = period * (-np:1:np);
y_mask = period * (-np:1:np);

addrect; # substrate
set("name", "substrate");
set("index",substrate_index);

# ## set("material", "sio2");
# if (mat_sub=="<Object defined dielectric>") {{
#     set("index",substrate_index);
# }}

set("x span",3*max(x_mask));
set("y span",3*max(x_mask));
set("z max",0);
set("z min",-2*{wavelength});

M=readdata(zemax_optimized_phase_file);
len= length(M)/3;
for(i=1:len) {{
    addcircle({{"name":"pillar", "x":M(i),"y":M(len+i),"radius":M(2*len+i)}});
}}
select("pillar");
set("index",index_pillar);
# set("material", "si");


set("z min",0);
set("z max",1.3e-6);
?height;
"""

fdtd.set("script", myscript)

# #==================================================
# Add the aperture
# #==================================================

fdtd.add2dpoly()
fdtd.set("name", "aperture")
fdtd.set("z", -0.05e-6)
R = LensDiameter/2
W = 3* 10e-6
apertureP= np.linspace(0, 2*np.pi, 200)
V = R*np.exp(complex(0,1)* apertureP)
#V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
#fdtd.set("vertices", "[1.0e-6, 2.0e-6; 1.5e-6,2.0e-6]")
#  setnamed("aperture","vertices",V);

# #==================================================
# Add the source 
# #==================================================
fdtd.addplane()
fdtd.set("name", "source")
fdtd.set("injection axis","z")
fdtd.set("direction","forward")
fdtd.set("x",0)
fdtd.set("x span",30.0e-6)

fdtd.set("y",0)
fdtd.set("y span",30.0e-6)

fdtd.set("z",-0.15e-6)

fdtd.set("center wavelength", 1.55e-6)
fdtd.set("wavelength span",0.0e-6)

# #==================================================
# Add the FIELD monitor
# #==================================================
fdtd.addpower()
fdtd.set("name","field")
fdtd.set("monitor type",7)  # 2D z-normal
fdtd.set("x",0)
fdtd.set("x span",25.0e-6)
fdtd.set("y",0)
fdtd.set("y span",25.0e-6)
fdtd.set("z",1.365e-6)
# #==================================================

# #==================================================
# Add the Transmission monitor
# #==================================================
fdtd.addprofile()
fdtd.set("name","transmission")
fdtd.set("monitor type",7)  # 2D z-normal
fdtd.set("x",0)
fdtd.set("x span",25.0e-6)
fdtd.set("y",0)
fdtd.set("y span",25.0e-6)
fdtd.set("z",1.365e-6)
fdtd.set("output Pz", True)

fdtd.setglobalmonitor("frequency points", 1) # setting the global frequency resolution
fdtd.save("lumerical_full_lens_test")
fdtd.run()

# #====================================================

# monitor_name="transmission"
# f=fdtd.getdata(monitor_name,"f")
# T1=transmission(mon_name)
# print("normalized power through surface near the metalens (using direct transmission): "+ num2str(T1));

# # second way to calculate power through a surface - transmission
# x=getdata(mon_name,"x");
# y=getdata(mon_name,"y");
# f=getdata(mon_name,"f");
# Pz=getdata(mon_name,"Pz");
# Pz=pinch(real(Pz));

# T2 = 0.5*integrate(Pz,1:2,x,y);  # power in watts
# T2 = T2/sourcepower(f);
# print("normalized power through surface near the metalens (by using Poynting vector): 










# ## Step 4: Create the structures

# addrect; # substrate
# set("name","substrate");
# set("material", "sio2");
# if (mat_sub=="<Object defined dielectric>") {
#     set("index",index_sub);
# }
# set("x span",3*max(x_mask));
# set("y span",3*max(x_mask));
# set("z max",0);
# set("z min",-2*wavelength);


# M=readdata("..//output_x_y_rad_apexPHAZE_optimized.txt");
# len= length(M)/3;
# for(i=1:len) {
#     addcircle({"name":"pillar", "x":M(i),"y":M(len+i),"radius":M(2*len+i)});
    
# }


# select("pillar");
# set("index",index_pillar);
# set("material", "si");


# set("z min",0);
# set("z max",1.3e-6);
# ?height;
# set("detail",min([0,%pillar rendering detail%]));
# fdtd.adduserprop("radius",2,0.5e-6)

# myscript =      "addcircle; \n"
# myscript = myscript + "copy(1e-6); \n"
# myscript = myscript + "selectall; \n"
# myscript = myscript + "set(\"radius\",radius);"
# fdtd.set("name","dimer")
# fdtd.set("script",myscript)

